In [10]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv("../data/data_imputed_3.csv")

In [12]:
df.head()

,timestamp_local,temp,aqi,co,no2,o3,pm10,pm25,so2
0,01/02/2022 0:00,12.6,385,1339.8,76.0,10.7,491.7,347.67,238.0
1,01/02/2022 1:00,11.5,404,1437.6,76.0,9.3,508.3,359.33,268.0
2,01/02/2022 2:00,11.9,421,1535.5,76.0,8.0,525.0,371.00,298.0
3,01/02/2022 3:00,12.2,425,1659.0,68.3,5.3,529.3,374.00,275.7
4,01/02/2022 4:00,11.9,430,1782.5,60.7,2.7,533.7,377.00,253.3


In [14]:
data = df[['pm25']]

In [15]:
from sklearn.preprocessing import MinMaxScaler
values = data.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [16]:
scaled = scaled.reshape((20978,))

In [17]:
scaled.shape

(20978,)

In [18]:
def df_to_X_y(df, window_size, forecast_length):
    # df_as_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df) - window_size - forecast_length):
        row_x = [[a] for a in df[i:i+window_size]]
        
        X.append(row_x)
        # row_y = df_as_np[i+window_size]
        row_y = [b for b in df[i+window_size:i+window_size+forecast_length]]
        
        y.append(row_y)
        
    return np.array(X), np.array(y)

In [19]:
def train_test_split(X, y, test_size=0.2):
    split_idx = int(len(X) * (1 - test_size))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    return X_train, X_test, y_train, y_test

In [21]:
WINDOW_WIDTH = 168
FORECAST_LENGTH = 168
X, y = df_to_X_y(scaled, WINDOW_WIDTH, FORECAST_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

X_max = X_train.max()
X_min = X_train.min()

In [22]:
batchSize = 16
from keras.models import Sequential, save_model, load_model
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

In [23]:
normalizer = Normalization()
normalizer.adapt(X_train)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

model = Sequential()
# 1D Convolutional layers
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# Flatten the output of the Conv layers
model.add(Flatten())
# model.add(LSTM(input_shape = X_train.shape[1:], units=86, return_sequences=True))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(GRU(units=86, return_sequences=True))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(GRU(units=100, return_sequences=True))
# model.add(LSTM(units=100, return_sequences=False))
# model.add(Dense(units=100,  activation='relu'))
# model.add(Dense(8, activation='relu'))

model.add(Dense(units=FORECAST_LENGTH, activation='linear'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()] , loss=MeanSquaredError())

history = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[rlrop])

# score = model.evaluate(X_test, y_test, verbose = 1)

C:\Users\Hassan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 166, 8)              │              32 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1328)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 168)                 │         223,272 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 223,304 (872.28 KB)

 Trainable params: 223,304 (872.28 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0070 - root_mean_squared_error: 0.0824 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0383 - learning_rate: 0.0010
Epoch 2/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0044 - root_mean_squared_error: 0.0662 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0376 - learning_rate: 0.0010
Epoch 3/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0042 - root_mean_squared_error: 0.0648 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0381 - learning_rate: 0.0010
Epoch 4/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0038 - root_mean_squared_error: 0.0614 - val_loss: 0.0017 - val_root_mean_squared_error: 0.0403 - learning_rate: 0.0010
Epoch 5/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0038 - root_mean_squared_error: 0.0619 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0380 - learning_rate: 0.0010
Epoch 6/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0039 - root_mean_squared_err

In [24]:
forecast = model.predict(X_test)

130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [25]:
poll = np.array(df["pm25"])

meanop = poll.mean()
stdop = poll.std()

y_test_true = y_test*stdop + meanop
testPredict = forecast*stdop + meanop

In [26]:
from sklearn.metrics import mean_squared_error

In [27]:
rmse = np.sqrt(mean_squared_error(y_test_true, testPredict))
print("Test RMSE ="  ,rmse)

Test RMSE = 2.4625733


In [28]:
# Calculate MAPE
mape = np.mean(np.abs((y_test_true - testPredict) / y_test_true)) * 100
print("Test MAPE =", mape)

Test MAPE = 2.037653513252735


In [29]:
# Calculate accuracy as 1 - MAPE
accuracy = 100 - mape
print("Accuracy =", accuracy, "%")

Accuracy = 97.96234648674726 %


In [30]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE
mae = mean_absolute_error(y_test_true, testPredict)
print("Test MAE =", mae)

Test MAE = 1.6729987


In [31]:
y_test_true.shape

(4129, 168)

In [32]:
testPredict.shape

(4129, 168)

In [33]:
import numpy as np

def calculate_segment_rmse(predictions, actuals, segment_length=1):
    num_segments = predictions.shape[1] // segment_length
    rmses = []
    
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        
        # Extract the segment for both predictions and actuals
        pred_segment = predictions[:, start_idx:end_idx]
        actual_segment = actuals[:, start_idx:end_idx]
        
        # Calculate RMSE for this segment
        segment_rmse = np.sqrt(mean_squared_error(actual_segment.flatten(), pred_segment.flatten()))
        rmses.append(segment_rmse)
    
    return rmses

In [34]:
segment_rmses = calculate_segment_rmse(testPredict, y_test_true)
for i, rmse in enumerate(segment_rmses):
    print(f"RMSE for segment {i + 1}: {rmse:.4f}")

RMSE for segment 1: 0.5189
RMSE for segment 2: 0.7907
RMSE for segment 3: 1.0164
RMSE for segment 4: 1.2049
RMSE for segment 5: 1.3493
RMSE for segment 6: 1.4607
RMSE for segment 7: 1.5267
RMSE for segment 8: 1.5755
RMSE for segment 9: 1.6071
RMSE for segment 10: 1.6403
RMSE for segment 11: 1.6631
RMSE for segment 12: 1.6847
RMSE for segment 13: 1.6987
RMSE for segment 14: 1.7109
RMSE for segment 15: 1.7206
RMSE for segment 16: 1.7287
RMSE for segment 17: 1.7365
RMSE for segment 18: 1.7401
RMSE for segment 19: 1.7384
RMSE for segment 20: 1.7380
RMSE for segment 21: 1.7406
RMSE for segment 22: 1.7456
RMSE for segment 23: 1.7557
RMSE for segment 24: 1.7682
RMSE for segment 25: 1.7957
RMSE for segment 26: 1.8338
RMSE for segment 27: 1.8779
RMSE for segment 28: 1.9153
RMSE for segment 29: 1.9583
RMSE for segment 30: 1.9977
RMSE for segment 31: 2.0206
RMSE for segment 32: 2.0489
RMSE for segment 33: 2.0712
RMSE for segment 34: 2.0940
RMSE for segment 35: 2.1116
RMSE for segment 36: 2.1250
R

In [35]:
forecast

array([[0.15286723, 0.14981377, 0.14454927, ..., 0.20705813, 0.21128932,
        0.21695286],
       [0.14471439, 0.13788176, 0.1346653 , ..., 0.21422195, 0.21790007,
        0.21995872],
       [0.13540128, 0.1320749 , 0.13438097, ..., 0.21944231, 0.22362861,
        0.21791601],
       ...,
       [0.02859327, 0.03241791, 0.03477338, ..., 0.0331745 , 0.03623867,
        0.03961613],
       [0.02928583, 0.03251471, 0.03335475, ..., 0.03540375, 0.03893528,
        0.04288445],
       [0.03048321, 0.03143714, 0.03285784, ..., 0.0382637 , 0.04244003,
        0.04708286]], dtype=float32)

In [36]:
testPredict

array([[85.39434 , 85.20054 , 84.866394, ..., 88.833885, 89.10244 ,
        89.461914],
       [84.87688 , 84.44321 , 84.23905 , ..., 89.28858 , 89.52203 ,
        89.652695],
       [84.28577 , 84.07464 , 84.22101 , ..., 89.61992 , 89.88563 ,
        89.52304 ],
       ...,
       [77.50658 , 77.74933 , 77.89883 , ..., 77.79735 , 77.99184 ,
        78.2062  ],
       [77.55053 , 77.75547 , 77.808784, ..., 77.93884 , 78.16299 ,
        78.41364 ],
       [77.626526, 77.68707 , 77.77725 , ..., 78.12036 , 78.38544 ,
        78.68012 ]], dtype=float32)

In [37]:
y_test_true

array([[85.79445 , 85.5795  , 85.36455 , ..., 93.31774 , 93.64017 ,
        93.96259 ],
       [85.5795  , 85.36455 , 85.14959 , ..., 93.64017 , 93.96259 ,
        94.28502 ],
       [85.36455 , 85.14959 , 86.116875, ..., 93.96259 , 94.28502 ,
        93.64017 ],
       ...,
       [77.33933 , 77.37588 , 77.41135 , ..., 78.23568 , 78.52157 ,
        78.808525],
       [77.37588 , 77.41135 , 77.55429 , ..., 78.52157 , 78.808525,
        78.843994],
       [77.41135 , 77.55429 , 77.6983  , ..., 78.808525, 78.843994,
        78.88054 ]], dtype=float32)

In [38]:
model.save("pm25_1dcnn.keras")